# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
# Import all modules
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
sns.set()

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## Question 

Perform statistical analysis to find out whether race has a significant impact on the <br>
rate of call backs (interview calls) for the resume.

In [22]:
# Exploratory Data Analysis
print(data.columns)
print(data.info())
print(data.race.unique())
print(data.call.unique())

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id

From above, it is evident that all the rows have a race and call-back associated.

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

## 1. What test is appropriate for this problem? Does CLT apply?

We can solve the above problem through two-sample z-test for proportions <br>
where the success proportion is defined as the fraction of people in each race <br>
which get a call-back.

In [43]:
w=data[data.race=='w']
b=data[data.race=='b']
w_c=w[w.call==1]
w_nc=w[w.call==0]
b_c=b[b.call==1]
b_nc=b[b.call==0]
print('People marked as White sounding:', len(w))
print('People marked as White sounding and got the call back:', len(w_c))
print('People marked as White sounding and did not get the  call back:', len(w_nc))
print('People marked as Black sounding:', len(b))
print('People marked as Black sounding and got the call back:', len(b_c))
print('People marked as Black sounding and did not get the call back:', len(b_nc))

People marked as White sounding: 2435
People marked as White sounding and got the call back: 235
People marked as White sounding and did not get the  call back: 2200
People marked as Black sounding: 2435
People marked as Black sounding and got the call back: 157
People marked as Black sounding and did not get the call back: 2278


The CLT can be applied becasue of the following <br>
(1) The number of instance of success and failure in the sample are all greater than 10. <br>
(2) The sample size is greater than 30 in all cases. <br>
(3) The sample was selected in random. <br>

Hence, we can apply Central Limit Theorem and assume that the mean of the proportitions <br>
in sample distributions will follow a normal distribution.

## 2. What are the null and alternate hypotheses?

In [ ]:
<b> Null Hypothesis </b> : The proportion for call-back from black and white sounding races are same.
<b> Alternate Hypothesis </b> : The proportion for call-back from black and white sounding races are not the same.

In [6]:
#w = data[data.race=='w']
#b = data[data.race=='b']

## 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [46]:
# Frequentist Statistics Approach
w_c_prop=len(w_c)/len(w)
print(w_c_prop)
b_c_prop=len(b_c)/len(b)
print(b_c_prop)
prop_diff= w_c_prop - b_c_prop
print(prop_diff)

0.09650924024640657
0.06447638603696099
0.032032854209445585


From this sample, it is observed that the proportion of call-back for a white sounding person <br>
is higher than the proportion of call-back for a black sounding person <br>
Now, lets check whether the difference is statistically significant. <br>

In [49]:
phat = (len(w_c) + len(b_c)) / (len(w) + len(b))
z = prop_diff / np.sqrt(phat * (1 - phat) * ((1 / len(w)) + (1 / len(b))))
pval = stats.norm.cdf(-z) * 2
print("z score:",z)
print("p value:", pval)

Z score: 4.108412152434346
P-value: 3.983886837585077e-05


In [57]:
# Bootstrap Approach: Permultation test

permutation_replicates = np.empty(100000)

w_array=w_c.call.values
b_array=b_c.call.values
diff_mean=np.mean(w_array)-np.mean(b_array)

for i in range(len(permutation_replicates)):
    permutation_samples = np.random.permutation(np.concatenate((w_array, b_array)))
    
    w_array_perm = permutation_samples[:len(w_array)]
    b_array_perm = permutation_samples[len(w_array):]
    
    permutation_replicates[i] = np.abs(np.mean(w_array_perm) - np.mean(b_array_perm))
    
p_value = np.sum(permutation_replicates > diff_mean) / float(len(permutation_replicates))
print("p value", p_value)

p value 0.0


<b>Chi Squared Approach. </b> 
We can also find any correlation between the race and call columns. <br> 
However, as Chi-squared test is based on frequency (not proportion), we will consider <br>
actual frequency of each combination of columns.  <br>

In [58]:
cont_table = pd.crosstab(index=data.call, columns=data.race)
chi2,pval,_,_ = stats.chi2_contingency(cont_table)
print('Chi-Squared Test Statistics:',chi2)
print('p value:',pval)

Chi-Squared Test Statistics: 16.44902858418937
p value: 4.997578389963255e-05


In [62]:
margin_error=1.96 * np.sqrt(phat * (1 - phat) * ((1 / len(w)) + (1 / len(b))))
ci = prop_diff + np.array([-1,1])* margin_error
print ("Margin of Error:", margin_error)
print("Confidence Interval:",ci)

Margin of Error: 0.015281912310894095
Confidence Interval: [0.01675094 0.04731477]


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

## 4. Write a story describing the statistical significance in the context of the  original problem.


<b>Conclusion</b>

As the p value is close to zero in all the 3 approach, we reject the null hypothesis <br>.
In other words, the call back rate for white sounding and black sounding people are statistically different.<br>

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, Our analysis just concluded that race is one of the factors that might influence <br>
the call back success. However, the analysis is not sufficient to conclude that it is <br>
the most important factor. There may be other factors which have a higher infleunce on <br>
the callback success or there may be interactions among variables as well. <br>